<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/notion_expense_sync_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ติดตั้งไลบรารีหลักที่ใช้:
# - gspread: ใช้เชื่อมกับ Google Sheets
# - oauth2client: สำหรับ auth กับ Google API
# - notion-client: ใช้ยิงข้อมูลเข้า Notion API
!pip install gspread oauth2client notion-client

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# STEP 1: ใส่ Google Service Account JSON ไฟล์ (upload เอง หรือใช้ secret manager)
# ตัวอย่างไฟล์ชื่อว่า 'service_account.json'

# ระบุ scope สำหรับใช้งาน Google Sheets และ Drive API
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/drive']

# auth และสร้าง client
creds = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
client = gspread.authorize(creds)

# เปิด Sheet โดยใช้ Sheet ID ที่พี่สร้างไว้
sheet = client.open_by_key("1j9GE9KsQnZdUVohsDJ9e97FDqQC-g1-Cv81L1olnHu4").sheet1

In [ ]:
# แปลงข้อความที่ส่งมาจาก Line เช่น "Food 180"
# → แยกเป็น category = "Food", amount = 180
def parse_message(text):
    try:
        parts = text.strip().split()
        category = parts[0]
        amount = float(parts[1])
        return category, amount
    except:
        # ถ้า format ไม่ถูกต้องจะคืน None
        return None, None

In [ ]:
from notion_client import Client

# เชื่อมกับ Notion โดยใช้ Token ที่พี่ได้มาจาก my-integrations
notion = Client(auth="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")

# ID ของ Notion Database ที่จะเขียนเข้า
database_id = "1be50a040a82806ea775ddc399cfbb45"

# ฟังก์ชันสำหรับสร้างรายการใหม่ใน Notion โดยใส่หมวดและยอดเงิน
def push_to_notion(category, amount):
    notion.pages.create(
        parent={"database_id": database_id},
        properties={
            "Categories": {"select": {"name": category}},
            "Amount": {"number": amount}
        }
    )


In [ ]:
# ฟังก์ชันหลักสำหรับ Sync:
# - อ่านทุกแถวใน Google Sheet
# - คัดเฉพาะแถวที่ยังไม่ Sync (Synced? == "NO")
# - แปลงข้อความ และยิงเข้า Notion
# - อัปเดตคอลัมน์ Synced? ให้เป็น "YES"

def sync_sheet_to_notion():
    data = sheet.get_all_records()
    for idx, row in enumerate(data):
        if row['Synced?'].strip().upper() != "NO":
            continue  # ข้ามแถวที่ sync แล้ว

        category, amount = parse_message(row['Message'])
        if not category or not amount:
            print(f"❌ ข้าม row {idx+2}: format ผิด")
            continue

        try:
            push_to_notion(category, amount)
            sheet.update_cell(idx+2, 4, "YES")  # update "Synced?" เป็น YES
            print(f"✅ Sync row {idx+2}: {category} {amount}")
        except Exception as e:
            print(f"🚨 Error at row {idx+2}: {e}")

# รันฟังก์ชัน
sync_sheet_to_notion()
